In [1]:
library(miloR)
library(igraph)
library(BiocParallel)
library(SingleCellExperiment)
library(Matrix)
library(dplyr)
setwd('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

Loading required package: edgeR

Loading required package: limma


Attaching package: ‘igraph’


The following object is masked from ‘package:miloR’:

    graph


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    c

In [2]:
load('h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo_prep.RData')
adata_no_knnx

class: SingleCellExperiment 
dim: 318 20714 
metadata(21): Interferon_group_colors WHO_temp_severity_colors ...
  seurat_clusters_colors umap
assays(2): X counts
rownames(318): HES4 MEGF6 ... AC233755.2 AC233755.1
rowData names(9): vst.mean vst.variance ... dispersions
  dispersions_norm
colnames(20714): UK-CIC10690315+UK-CIC10690507_AACCGCGCACTCTGTC-1
  UK-CIC10690315+UK-CIC10690507_AACGTTGAGGGCATGT-1 ...
  UK-CIC10690384+UK-CIC10690576_TTTGGTTGTACGAAAT-1
  UK-CIC10690384+UK-CIC10690576_TTTGGTTGTTGTCTTT-1
colData names(67): orig.ident nCount_RNA ... time_from_positive_swab
  time_from_infection
reducedDimNames(6): X_harmony_rna X_pca_rna ... X_umapafterharmony_rna
  X_umapbeforeharmony_rna
altExpNames(0):

In [3]:
mylo <- Milo(adata_no_knnx)
milo_graph <- buildFromAdjacency(knn_adjacencyx, k=50, is.binary=TRUE)
miloR::graph(mylo) <- miloR::graph(milo_graph)

In [4]:
dim(reducedDims(mylo)[['X_scVI']])

[1] 20714    10

In [5]:
mylo <- buildGraph(mylo, k=50, d=10, reduced.dim="X_scVI", BPPARAM = MulticoreParam(progressbar = TRUE))
mylo <- makeNhoods(mylo, prop = 0.3, k=50, d=10, reduced_dim="X_scVI")

  |======================================================================| 100%



Constructing kNN graph with k:50

Checking valid object



In [6]:
mylo <- countCells_(mylo, meta.data = data.frame(colData(mylo)), samples="sample_id")
mylo <- calcNhoodDistance(mylo, d=10, reduced.dim = 'X_scVI', use.assay = 'counts')
mylo <- buildNhoodGraph(mylo)
saveRDS(mylo, 'h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo.RDS')

Checking meta.data validity

Counting cells in neighbourhoods



In [7]:
# kernel died...
library(miloR)
library(igraph)
library(BiocParallel)
library(SingleCellExperiment)
library(Matrix)
library(dplyr)
setwd('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [8]:
mylo <- readRDS('h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo.RDS')
metadata = data.frame(colData(mylo))

In [9]:
# metadata$case_control <- ordered(factor(metadata$case_control, levels = c('NEGATIVE', 'POSITIVE', 'RECOVERY')))
metadata$case_control <- factor(metadata$case_control, levels = c('NEGATIVE', 'POSITIVE', 'RECOVERY'))
metadata$WHO_temp_severity <- factor(metadata$WHO_temp_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'))
metadata$WHO_temp_severity_group <- factor(metadata$WHO_temp_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'), labels = c('NA', 'mild_moderate', 'mild_moderate', 'severe_critical', 'severe_critical'))
metadata$WHO_severity_group <- factor(metadata$WHO_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'), labels = c('NA', 'mild_moderate', 'mild_moderate', 'severe_critical', 'severe_critical')) # interpreted as peak severity
metadata$grouped_temp_severity <- ifelse(metadata$WHO_temp_severity %in% c("mild", "moderate"), "mild_moderate", "severe_critical")
metadata$grouped_severity <- ifelse(metadata$WHO_severity %in% c("mild", "moderate"), "mild_moderate", "severe_critical")
metadata$age_scaled <- scale(metadata$calc_age) # scale age
metadata$ethnicity = factor(metadata$ethnicity)
metadata$sex = factor(metadata$sex)
metadata$centre = factor(metadata$centre)

In [10]:
# remove 
metadata <- distinct(metadata[, c('sample_id', 'individual_id', 'case_control', 'WHO_temp_severity', 'WHO_temp_severity_group', 'WHO_severity_group', 'grouped_temp_severity', 'grouped_severity', 'sex', 'ethnicity', 'age_scaled', 'centre')], .keep_all = TRUE)
metadata

,sample_id,individual_id,case_control,WHO_temp_severity,WHO_temp_severity_group,WHO_severity_group,grouped_temp_severity,grouped_severity,sex,ethnicity,age_scaled,centre
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<fct>,<fct>,"<dbl[,1]>",<fct>
UK-CIC10690315+UK-CIC10690507_AACCGCGCACTCTGTC-1,C137_280121,C137,POSITIVE,mild,mild_moderate,mild_moderate,mild_moderate,mild_moderate,F,asian,0.4296535,Cambridge
UK-CIC10690315+UK-CIC10690507_AACGTTGAGGGCATGT-1,C140_280121,C140,POSITIVE,mild,mild_moderate,mild_moderate,mild_moderate,mild_moderate,F,asian,-1.3489883,Cambridge
UK-CIC10690317+UK-CIC10690509_AAACGGGAGCGATTCT-1,C140_300121,C140,POSITIVE,mild,mild_moderate,mild_moderate,mild_moderate,mild_moderate,F,asian,-1.3489883,Cambridge
UK-CIC10690321+UK-CIC10690513_AAAGATGGTAGCTGCC-1,C140_110321,C140,RECOVERY,NA,NA,NA,severe_critical,severe_critical,F,asian,-1.3489883,Cambridge
UK-CIC10690321+UK-CIC10690513_AAATGCCGTGGCAAAC-1,C137_250221,C137,RECOVERY,NA,NA,NA,severe_critical,severe_critical,F,asian,0.4296535,Cambridge
UK-CIC10690325+UK-CIC10690517_AAACCTGGTTAAGTAG-1,C140_020221,C140,POSITIVE,mild,mild_moderate,mild_moderate,mild_moderate,mild_moderate,F,asian,-1.3489883,Cambridge
UK-CIC10690325+UK-CIC10690517_AAACGGGAGAACTGTA-1,C137_300420,C137,NEGATIVE,NA,NA,NA,severe_critical,severe_critical,F,asian,0.4296535,Cambridge
UK-CIC10690325+UK-CIC10690517_AAAGCAAAGGATCGCA-1,C146_250221,C146,RECOVERY,NA,NA,NA,severe_critical,severe_critical,M,asian,0.7180819,Cambridge
UK-CIC10690329+UK-CIC10690521_AAAGTAGAGGGTTCCC-1,C146_270121,C146,POSITIVE,moderate,mild_moderate,severe_critical,mild_moderate,severe_critical,M,asian,0.7180819,Cambridge


In [11]:
library(lmerTest)
library(pbmcapply)

Loading required package: lme4


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [12]:
res <- as.list(1:nrow(nhoodCounts(mylo)))
res <- pbmclapply(res, function(x) glmer.nb(nhoodCounts(mylo)[x,] ~ case_control + sex + ethnicity + age_scaled + (1|individual_id), offset = log(colSums(nhoodCounts(mylo))), data = metadata, control=glmerControl(check.conv.singular = .makeCC(action = "ignore",  tol = 1e-2), optimizer="bobyqa", tol=1e-02)), mc.cores = parallel::detectCores())
res_summary <- pbmclapply(res, summary)
save(res_summary, res, metadata, file = 'h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo_results.RData')

In [13]:
# example coefficients for 1 neighborhood
res_summary[[1]]$coefficients

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-23.8312240,482.7182592,-0.04936881,0.960625387
case_controlPOSITIVE,15.1005107,482.7183811,0.03128224,0.975044456
case_controlRECOVERY,12.6135167,482.7194652,0.02613012,0.979153554
sexM,1.3852260,0.7091449,1.95337518,0.050775156
ethnicityother,-1.1283237,1.0249852,-1.10081949,0.270975226
ethnicitywhite,-3.9042341,1.2616187,-3.09462284,0.001970633
age_scaled,0.1841658,0.4019664,0.45816210,0.646835987


In [55]:
singulars <- lapply(res, function(x) as.numeric(isSingular(x)))
conv <- lapply(res, function(x) length(slot(x, "optinfo")$conv$lme4$messages))

In [56]:
length(singulars)

[1] 5067

In [58]:
length(conv)

[1] 5067

In [14]:
pvalue <- pbmclapply(res_summary, function(x) x$coefficients[3,4])
beta <- pbmclapply(res_summary, function(x) x$coefficients[3,1])

In [15]:
da.res = data.frame(Nhood = as.numeric(1:nrow(nhoodCounts(mylo))), beta = do.call(c, beta), PValue = do.call(c, pvalue))

In [59]:
dim(da.res)

[1] 5067    7

In [60]:
da.res$Singular <- singulars
da.res$Conv <- conv

In [70]:
da.res$SpatialFDR <- graphSpatialFDR(x.nhoods=nhoods(mylo), graph=miloR::graph(mylo),
    weighting='k-distance', pvalues=da.res$PValue,
    indices=nhoodIndex(mylo), distances=nhoodDistances(mylo),
    reduced.dimensions=reducedDim(mylo, 'X_scVI'), k = 50)
da.res$Diff <- sign(da.res$beta)
da.res$Diff[da.res$SpatialFDR >= 0.1 | da.res$Singular > 0 | da.res$Conv > 0] <- 0
saveRDS(da.res, file = 'h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo_results_DA.RDS')

In [78]:
table(da.res$Diff)


  -1    0 
   1 5066 

In [133]:
# da.res <- readRDS('h5ad/df.fil3_gex_bcells_no_CLL_vdj_trimmed_wave2_rec_vs_neg_milo_results_DA.RDS')

In [134]:
# library(ggbeeswarm)
# library(ggplot2)
# new_DA_beeswarm <- function (da.res1, da.res2 = NULL, max = 5, color.by = 'SpatialFDR', group.by = NULL, split.by =NULL, alpha = 0.1, subset.nhoods = NULL, size = 2, filter = c('Mixed', 'doublets')) {
#     if (!is.null(da.res2) && !is.null(split.by)){
#         da.res1$split = split.by[1]
#         da.res2$split = split.by[2]
#         da.res <- rbind(da.res1, da.res2)
#     } else {
#         da.res <- da.res1
#     }

#     da.res$beta[da.res$beta > max] <- max
#     da.res$beta[da.res$beta < -max] <- -max
#     da.res <- da.res %>% filter(!get(group.by) %in% filter)
#     if (!is.null(group.by)) {
#         if (!group.by %in% colnames(da.res)) {
#             stop(paste0(group.by, " is not a column in da.res. Have you forgot to run annotateNhoods(x, da.res, ", 
#                 group.by, ")?"))
#         }
#         if (is.numeric(da.res[, group.by])) {
#             stop(paste0(group.by, " is a numeric variable. Please bin to use for grouping."))
#         }
#         da.res <- mutate(da.res, group_by = da.res[, group.by])
#     }else {
#         da.res <- mutate(da.res, group_by = "g1")
#     }    
#     if (!is.factor(da.res[, "group_by"])) {
#         message(paste0("Converting group.by to factor..."))
#         da.res <- mutate(da.res, factor(group_by, levels = unique(group_by)))
#     }
#     if (!is.null(subset.nhoods)) {
#         da.res <- da.res[subset.nhoods, ]
#     }
    
#     da.res <- da.res %>% mutate(is_signif = ifelse((get(color.by) < alpha & get('Singular') < 1 & get('Conv') < 1), 
#         1, 0)) %>% mutate(beta_color = ifelse(is_signif == 1, 
#         beta, NA)) %>% arrange(group_by) %>% mutate(Nhood = factor(Nhood, 
#         levels = unique(Nhood)))
#     if (!is.null(split.by)){
#         p <- ggplot(da.res, aes(group_by, beta, color = beta_color)) + 
#             scale_color_gradient2(midpoint = 0, low = "#0571b0", mid = '#f7f7f7', high = "#ca0020", limits=c(-max/3, max/3), oob=scales::squish) +
#             guides(color = "none") + 
#             xlab(group.by) + 
#             ylab("beta") + 
#             geom_quasirandom(alpha = 1, size = size) +
#             coord_flip() + 
#             theme_bw() + 
#             theme(axis.title.y = element_blank(),
#                   axis.title.x = element_text(size=20, color = '#000000'),
#                   axis.text = element_text(size=20, color = '#000000'),
#                   axis.ticks = element_line(color = '#000000'),
#                   panel.border = element_rect(colour = "#000000")
#                 ) + 
#             facet_wrap(split~.) +
#             theme(strip.background = element_blank(), strip.text.x = element_text(angle = 0, size = 20, color = '#000000'))
#     } else {
#         p <- ggplot(da.res, aes(group_by, beta, color = beta_color)) + 
#         scale_color_gradient2(midpoint = 0, low = "#0571b0", mid = '#f7f7f7', high = "#ca0020", limits=c(-max/3, max/3), oob=scales::squish) +
#         guides(color = "none") + 
#         xlab(group.by) + 
#         ylab("beta") + 
#         geom_quasirandom(alpha = 1, size = size) +
#         coord_flip() + 
#         theme_bw() + 
#         theme(axis.title.y = element_blank(),
#               axis.title.x = element_text(size=20, color = '#000000'),
#               axis.text = element_text(size=20, color = '#000000'),
#               axis.ticks = element_line(color = '#000000'),
#               panel.border = element_rect(colour = "#000000")
#             ) + 
#         theme(strip.background = element_blank(), strip.text.x = element_text(angle = 0, size = 20, color = '#000000'))
#     }
    
#     p$data <- p$data[order(p$data$beta_color, na.last = FALSE), ]
#     return(p)
# }

In [131]:
# options(repr.plot.width=20, repr.plot.height=10)
# da.res <- annotateNhoods(mylo, da.res, coldata_col = "celltype_B_corrected")
# da.res$celltype_B_corrected <- ifelse(da.res$celltype_B_corrected_fraction < 0.7, "Mixed", da.res$celltype_B_corrected)
# p <- new_DA_beeswarm(da.res, color.by = 'SpatialFDR', group.by = "celltype_B_corrected", size = 2, alpha = .05)
# p

In [132]:
# options(repr.plot.width=20, repr.plot.height=10)
# da.res <- annotateNhoods(mylo, da.res, coldata_col = "celltype_B_corrected_IFN")
# da.res$celltype_B_corrected_IFN <- ifelse(da.res$celltype_B_corrected_IFN_fraction < 0.7, "Mixed", da.res$celltype_B_corrected_IFN)
# p <- new_DA_beeswarm(da.res, color.by = 'SpatialFDR', group.by = "celltype_B_corrected_IFN", size = 2, alpha = .05)
# p